In [1]:
!pip install ludwig[llm]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.4 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 89.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 88.2 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.0/232.0 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 63.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 14.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import login

login()

# Load Dataset

In [4]:
from datasets import load_dataset

dataset = load_dataset("iamtarun/python_code_instructions_18k_alpaca", split="train")

In [5]:
import pandas as pd
df = pd.DataFrame(dataset)
df['input'] = df['input'].replace('Not applicable','')
df['input'] = df['input'].replace('No input','')
df

,instruction,input,output,prompt
0,Create a function to calculate the sum of a se...,"[1, 2, 3, 4, 5]",# Python code\ndef sum_sequence(sequence):\n ...,Below is an instruction that describes a task....
1,Generate a Python code for crawling a website ...,website: www.example.com \ndata to crawl: phon...,import requests\nimport re\n\ndef crawl_websit...,Below is an instruction that describes a task....
2,Create a Python list comprehension to get the ...,,"[x*x for x in [1, 2, 3, 5, 8, 13]]",Below is an instruction that describes a task....
3,Generate a python script to perform this action.,"Given a string, remove all the consecutive dup...",def remove_duplicates(string): \n result = ...,Below is an instruction that describes a task....
4,Write a python script to generates random numb...,,def generate_random_divisible_number():\n i...,Below is an instruction that describes a task....
...,...,...,...,...
18607,Generate a loop in Python which iterates throu...,,for value in list_of_values:\n print(value),Below is an instruction that describes a task....
18608,Can you design a function in Python to sort a ...,"tuples = [(""b"", 0), (""a"", 1), (""c"", -1)]",def sort_tuple(tuples):\n # sort by 2nd ite...,Below is an instruction that describes a task....
18609,Generate a Python function to search an array ...,"Array = [1, 5, 10, 15, 20], Target = 15","def search(array, target):\n # Iterate over ar...",Below is an instruction that describes a task....
18610,Develop an HTTP API in Python to retrieve the ...,,import requests\nfrom bs4 import BeautifulSoup...,Below is an instruction that describes a task....


# Training Config

In [6]:
import yaml

config_str = """
model_type: llm
base_model: deepseek-ai/deepseek-coder-6.7b-base

adapter:
    type: lora

quantization:
    bits: 4

prompt:
  template: |
    Complete the Task bellow with code:

    ### Task: {instruction}
    {input}

    ### Code:
    
input_features:    
  - name: instruction
    type: text
    preprocessing:
      max_sequence_length: 512

    
output_features:
  - name: output
    type: text
    preprocessing:
      max_sequence_length: 512

generation:
  max_new_tokens: 512
  temperature: 0.1

trainer:
  type: finetune
  batch_size: 1
  gradient_accumulation_steps: 16
  epochs: 3
  learning_rate: 0.0004
  learning_rate_scheduler:
    warmup_fraction: 0.02
    
preprocessing:
  global_max_sequence_length: 1024
  split:
    probabilities:
      - 0.8
      - 0.1
      - 0.1
  
"""

config = yaml.safe_load(config_str)

In [7]:
import logging
from ludwig.api import LudwigModel

model = LudwigModel(config=config, logging_level=logging.INFO)
results = model.train(dataset=df)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='linear': {'type'}
Unrecognized keys in `rope_scaling` for 'rope_type'='linear': {'type'}


tokenizer_config.json:   0%|          | 0.00/793 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

...

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


trainable params: 4,194,304 || all params: 6,744,707,072 || trainable%: 0.0622
Evaluation valid:   0%|          | 0/931 [00:00<?, ?it/s] 1.07it/s, loss=0.00934] 

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Training: 100%|██████████| 44670/44670 [9:09:00<00:00,  1.36it/s, loss=0.0446]


Unrecognized keys in `rope_scaling` for 'rope_type'='linear': {'type'}
Unrecognized keys in `rope_scaling` for 'rope_type'='linear': {'type'}


In [8]:
!ludwig upload hf_hub --repo_id JVictor-CC/deepseek-coder-6.7b-final-2 --model_path /kaggle/working/results/api_experiment_run

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


adapter_model.safetensors: 100%|███████████| 16.8M/16.8M [00:01<00:00, 11.1MB/s]
